In [1]:
import os,random
import csv
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import numpy as np
from copy import deepcopy
from tensorflow.keras.models import load_model
import tensorflow.keras.models as models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, AlphaDropout, GlobalAveragePooling1D, BatchNormalization, add
from tensorflow.python.keras.layers.core import Flatten, Dense, Dropout, Reshape, Activation,Lambda,Permute
from tensorflow.python.keras.layers.convolutional import Conv1D,Conv2D, AveragePooling1D, ZeroPadding2D, Convolution2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.regularizers import *
from tensorflow.keras import layers
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import pickle,time
import random, sys
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
mirrored_strategy = tf.distribute.MirroredStrategy()
from art.attacks.evasion import CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import KerasClassifier
tf.compat.v1.disable_eager_execution()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
# Dataset setup
Xd = pickle.load(open('RML22.pickle','rb'))

snrs,mods = map(lambda j: sorted( list( set( map( lambda x: x[j], Xd.keys() ) ) ) ), [1,0])

X = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)][0:6000])
        for i in range(6000):  lbl.append((mod,snr))
X = np.vstack(X)
print(X.shape)

(1260000, 2, 128)


In [3]:
def to_onehot(yin):
    yy = list(yin)
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_snr = to_onehot(map(lambda x: mods.index(lbl[x][0]), range(X.shape[0])))

In [4]:
# Partition the dataset into training and testing datasets
np.random.seed(2022)
n_examples = X.shape[0]
n_train    = int(round(n_examples * 0.5))
train_idx  = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx   = list(set(range(0,n_examples))-set(train_idx))
X_train    = X[train_idx]
X_test     = X[test_idx]

Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

input_shp = list(X_train.shape[1:])
output_shp = Y_train.shape[1]
print(f'Network input shape in variable input-shp: {input_shp}')
print(f'Network output shape in variable input-shp: {output_shp}')
print(X_train.shape, input_shp)
classes = mods

Network input shape in variable input-shp: [2, 128]
Network output shape in variable input-shp: 10
(630000, 2, 128) [2, 128]


In [5]:
X_train_orig = deepcopy(X_train)
X_test_orig = deepcopy(X_test)

num_samples_list = [16,32,64]
field_names= ['num_subsample'] + list(snrs)

In [6]:
# versions
targetModels = ["complex_cnn", "resnet", "cldnn", "holistic", "uniform"]

In [ ]:
for targetModel in targetModels:
    def residual_stack(x):
        def residual_unit1(y,_strides=1):
            shortcut_unit=y
            # 1x1 conv linear
            y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)

            y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            # add batch normalization
            y = layers.add([shortcut_unit,y])
            return y
    
        def residual_unit2(y,_strides=1):
            shortcut_unit=y
            # 1x1 conv linear
            y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                          kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            # add batch normalization
            y = layers.add([shortcut_unit,y])
            return y
    
        def residual_unit3(y,_strides=1):
            shortcut_unit=y
            # 1x1 conv linear
            y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            # add batch normalization
            y = layers.add([shortcut_unit,y])
            return y
    
        def residual_unit4(y,_strides=1):
            shortcut_unit=y
            # 1x1 conv linear
            y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
            y = layers.BatchNormalization()(y)
            # add batch normalization
            y = layers.add([shortcut_unit,y])
            return y
    
        x = layers.Conv1D(8, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                      kernel_initializer='glorot_uniform')(x)
        x = layers.BatchNormalization()(x)

        x = residual_unit1(x)
        x = layers.Conv1D(16, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
        x = layers.BatchNormalization()(x)
        x = residual_unit2(x)
        x = layers.Conv1D(32, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
        x = layers.BatchNormalization()(x)
        x = residual_unit3(x)
        x = layers.Conv1D(64, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
        x = layers.BatchNormalization()(x)
        x = residual_unit4(x)
        x = layers.AveragePooling1D(data_format='channels_first',pool_size=in_shp[1])(x)
        return x

    acc_list = []
    acc_attack_list = []

    for num_samples in num_samples_list:

        X_train = deepcopy(X_train_orig)
        X_test =  deepcopy(X_test_orig)

        X_train_sub = np.zeros((X_train.shape[0], X_train.shape[1], num_samples))
        X_test_sub = np.zeros((X_test.shape[0], X_test.shape[1], num_samples))


        X_train = X_train_sub
        X_test = X_test_sub
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        print(X_train.shape)
        print(Y_train.shape)

        in_shp = list(X_train.shape[1:])
        print(X_train.shape, in_shp, snrs)
        classes = mods

        inputs=layers.Input(shape=in_shp)
        x = residual_stack(inputs)
        x = Flatten()(x)
        x = Dense(256,kernel_initializer="he_normal", activation="selu", name="dense1")(x)
        x = AlphaDropout(0.5)(x)
        x = Dense(len(classes),kernel_initializer="he_normal", activation="softmax", name="dense3")(x)
        model = models.Model(inputs,x)
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
        model.summary()


        nb_epoch = 200     # number of epochs to train on
        batch_size = 512  # training batch size

        filepath = './data/model/subsample_' + targetModel + '_' + str(num_samples) + '.wts.h5'
        model.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adam(lr=0.001),
                    metrics=['accuracy'])

        model.load_weights(filepath)

        SNR_vec =  np.arange(-20, 21, 2)
        X_train = deepcopy(X_train_orig)
        X_test =  deepcopy(X_test_orig)

        acc_attack_cnn = {}
        acc_attack_cldnn = {}
        acc_attack_holistic = {}
        acc_attack_resnet = {}
        acc_attack_uniform = {}

        acc_attack_cnn['num_subsample'] = 'complex_cnn'
        acc_attack_cldnn['num_subsample'] = 'cldnn'
        acc_attack_holistic['num_subsample'] = 'holistic'
        acc_attack_resnet['num_subsample'] = 'resnet'
        acc_attack_uniform['num_subsample'] = 'uniform'

        startIdx = 0
        for i, SNR in enumerate(snrs):
            #==============================================================================
            print('SNR = ', SNR)
            # # Here we load the DATA for modulation classification
    ###############################################################################################################
            with open('./data/resnet_index/indexList_' + str(SNR)+ '.data', 'rb') as fp:
                resnet_list = pickle.load(fp)
            with open('./data/complex_cnn_index/indexList_' + str(SNR)+ '.data', 'rb') as fp:
                cnn_list = pickle.load(fp)
            with open('./data/cldnn_index/indexList_' + str(SNR)+ '.data', 'rb') as fp:
                cldnn_list = pickle.load(fp)

            holistic_list = []
            snr_idx_uniform = list(range(1, 128, int(128/num_samples)))

            cldnn_list.sort(key=lambda x: x[1])
            cldnn_list = cldnn_list[:num_samples]
            snr_idx_cldnn = [ele[0] for ele in cldnn_list]
            snr_idx_cldnn.sort()
            cldnn_dict = {k: v for k, v in cldnn_list}

            cnn_list.sort(key=lambda x: x[1])
            cnn_list = cnn_list[:num_samples]
            snr_idx_cnn = [ele[0] for ele in cnn_list]
            snr_idx_cnn.sort()
            cnn_dict = {k: v for k, v in cnn_list}

            resnet_list.sort(key=lambda x: x[1])
            resnet_list = resnet_list[:num_samples]
            snr_idx_resnet = [ele[0] for ele in resnet_list]
            snr_idx_resnet.sort()
            resnet_dict = {k: v for k, v in resnet_list}

            cldnn_cnn_list = [(k, cldnn_dict[k] + cnn_dict[k]) for k in set(cldnn_dict).intersection(cnn_dict)]
            cldnn_cnn_dict = {k: v for k, v in cldnn_cnn_list}
            cnn_resnet_list = [(k, cnn_dict[k] + resnet_dict[k]) for k in set(cnn_dict).intersection(resnet_dict)]
            cnn_resnet_dict = {k: v for k, v in cnn_resnet_list}
            cldnn_resnet_list = [(k, cldnn_dict[k] + resnet_dict[k]) for k in set(cldnn_dict).intersection(resnet_dict)]
            cldnn_resnet_dict = {k: v for k, v in cldnn_resnet_list}

            cldnn_cnn_resnet_list = [(k, cldnn_cnn_dict[k] + resnet_dict[k]) for k in set(cldnn_cnn_dict).intersection(resnet_dict)]
            cldnn_cnn_resnet_list.sort(key=lambda x: x[1])

            tier_1_samples = cldnn_cnn_resnet_list
            tier_1 = [ele[0] for ele in tier_1_samples]

            for ele in tier_1:
                del cldnn_cnn_dict[ele]
                del cnn_resnet_dict[ele]
                del cldnn_resnet_dict[ele]

            cldnn_cnn_list = [(k, v) for k, v in cldnn_cnn_dict.items()]
            cnn_resnet_list = [(k, v) for k, v in cnn_resnet_dict.items()]
            cldnn_resnet_list = [(k, v) for k, v in cldnn_resnet_dict.items()]

            tier_2_samples = cldnn_cnn_list + cnn_resnet_list + cldnn_resnet_list
            tier_2_samples.sort(key=lambda x: x[1])

            tier_2 = [ele[0] for ele in tier_2_samples]

            for ele in tier_1:
                del cldnn_dict[ele]
                del cnn_dict[ele]
                del resnet_dict[ele]

            for ele in tier_2:
                try: del cldnn_dict[ele]
                except: pass
                try: del cnn_dict[ele]
                except: pass
                try: del resnet_dict[ele]
                except: pass

            cldnn_list = [(k, v) for k, v in cldnn_dict.items()]
            cnn_list = [(k, v) for k, v in cnn_dict.items()]
            resnet_list = [(k, v) for k, v in resnet_dict.items()]

            tier_3_samples = cldnn_list + cnn_list + resnet_list
            tier_3_samples.sort(key=lambda x: x[1])

            holistic_list = holistic_list + tier_1_samples
            holistic_list = holistic_list + tier_2_samples
            holistic_list = holistic_list + tier_3_samples
            holistic_list = holistic_list[:num_samples]
            holistic_list.sort(key=lambda x: x[1])
            holistic_list.sort(key=lambda x: x[0])

            snr_idx_holistic = [ele[0] for ele in holistic_list]
            snr_idx_holistic.sort()
    ##################################################################################

            #TEST
            test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
            test_X_i = X_test[np.where(np.array(test_SNRs)== SNR)]
            test_Y_i = Y_test[np.where(np.array(test_SNRs)== SNR)]    

            def makeBackBoxExample(x_test_adv, attacked_idx, target_idx):
                x_test_adv_128 = deepcopy(test_X_i)
                x_test_adv_128[:,:,attacked_idx] = x_test_adv
                x_test_adv_64 = deepcopy(x_test_adv_128[:,:,target_idx])
                return x_test_adv_128, x_test_adv_64

            targetIDX = []
            if targetModel == 'complex_cnn' : targetIDX = snr_idx_cnn
            if targetModel == 'cldnn' : targetIDX = snr_idx_cldnn
            if targetModel == 'resnet' : targetIDX = snr_idx_resnet
            if targetModel == 'holistic' : targetIDX = snr_idx_holistic
            if targetModel == 'uniform' : targetIDX = snr_idx_uniform

            x_test_adv_cnn = np.load('./data/attacked_' + str(num_samples) + '/attacked_'+str(num_samples)+'_' + str(SNR) + 'complex_cnn.npy')
            x_test_adv_cnn_128, x_test_adv_cnn_64 = makeBackBoxExample(x_test_adv_cnn, snr_idx_cnn, targetIDX)
            
            x_test_adv_cldnn = np.load('./data/attacked_' + str(num_samples) + '/attacked_'+str(num_samples)+'_' + str(SNR) + 'cldnn.npy')
            x_test_adv_cldnn_128, x_test_adv_cldnn_64 = makeBackBoxExample(x_test_adv_cldnn, snr_idx_cldnn, targetIDX)

            x_test_adv_holistic = np.load('./data/attacked_' + str(num_samples) + '/attacked_'+str(num_samples)+'_' + str(SNR) + 'holistic.npy')
            x_test_adv_holistic_128, x_test_adv_holistic_64 = makeBackBoxExample(x_test_adv_holistic, snr_idx_holistic, targetIDX)

            x_test_adv_resnet = np.load('./data/attacked_' + str(num_samples) + '/attacked_'+str(num_samples)+'_' + str(SNR) + 'resnet.npy')
            x_test_adv_resnet_128, x_test_adv_resnet_64 = makeBackBoxExample(x_test_adv_resnet, snr_idx_resnet, targetIDX)

            x_test_adv_uniform = np.load('./data/attacked_' + str(num_samples) + '/attacked_'+str(num_samples)+'_' + str(SNR) + 'uniform.npy')
            x_test_adv_uniform_128, x_test_adv_uniform_64 = makeBackBoxExample(x_test_adv_uniform, snr_idx_uniform, targetIDX)

            def makeEvaluation(x_test_adv, acc_given):
                test_Y_i_hat = model.predict(x_test_adv, batch_size=batch_size)
                conf = np.zeros([len(classes),len(classes)])
                confnorm = np.zeros([len(classes),len(classes)])
                for i in range(0,x_test_adv.shape[0]):
                    j = list(test_Y_i[i,:]).index(1)
                    k = int(np.argmax(test_Y_i_hat[i,:]))
                    conf[j,k] = conf[j,k] + 1
                for i in range(0,len(classes)):
                    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])   
                cor = np.sum(np.diag(conf))
                ncor = np.sum(conf) - cor
                print ("Overall Accuracy: ", cor / (cor+ncor))
                acc_given[SNR] = 1.0*cor/(cor+ncor)
            
            print('CNN      ', end='')
            makeEvaluation(x_test_adv_cnn_64, acc_attack_cnn)
            print('CLDNN    ', end='')
            makeEvaluation(x_test_adv_cldnn_64, acc_attack_cldnn)
            print('Resnet   ', end='')
            makeEvaluation(x_test_adv_resnet_64, acc_attack_resnet)
            print('Holistic ', end='')
            makeEvaluation(x_test_adv_holistic_64, acc_attack_holistic)
            print('Uniform  ', end='')
            makeEvaluation(x_test_adv_uniform_64, acc_attack_uniform)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_CNN.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_cnn)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_CLDNN.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_cldnn)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_ResNet.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_resnet)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_HOLISTIC.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_holistic)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_uniform.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_uniform)

        with open('./results/acc_' + str(num_samples) + '_attacked_' + targetModel + '_by_all.csv', 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=field_names)
            writer.writeheader()
            writer.writerow(acc_attack_cnn)
            writer.writerow(acc_attack_cldnn)
            writer.writerow(acc_attack_resnet)
            writer.writerow(acc_attack_holistic)
            writer.writerow(acc_attack_uniform)